In [ ]:
%matplotlib inline
from jupyterthemes import jtplot
jtplot.style(theme="onedork", context="notebook", ticks=True, grid=True)

import numpy as np
import itertools
import h5py
import os
import sys
import glob

import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns
import pandas as pd
import tikzplotlib

# from tqdm import tqdm
from tqdm.notebook import tqdm_notebook as tqdm

import helper.circular
# import fastpli.io
# import fastpli.tools
# import fastpli.objects
# import fastpli.analysis

In [ ]:
hist_bin = lambda n: np.linspace(0, np.pi, n + 1, endpoint=True)

df = pd.read_pickle(os.path.join("output/vs_1/voxel_size_simulation.pkl"))

fig, axs = plt.subplots(len(df.omega.unique()),
                        len(df.psi.unique()),
                        subplot_kw={'projection': 'polar'},
                        figsize=(20, 20))
axs = np.atleast_2d(axs)

for i, omega in enumerate(sorted(df.omega.unique())):
    for j, psi in enumerate(sorted(df.psi.unique())):
        axs[i, j].set_xticklabels([])
        axs[i, j].set_yticklabels([])
        axs[i, j].set_title(f"psi:{psi:.1f}, omega:{omega:.1f}")

for i, omega in enumerate(sorted(df.omega.unique())):
    for j, psi in enumerate(sorted(df.psi.unique())):
        sub = (df.omega
               == omega) & (df.psi == psi) & (df.voxel_size == 0.005) & (
                   df.f0_inc == 10.0) & (df.f1_rot == 0.0) & (df.model == "r")

        df_sub = df[sub]
        
        if len(df_sub) > 1:
            print("fooo", omega, psi)
        if len(df_sub) == 0:
            continue

        phi = df_sub.explode("f_phi").f_phi.to_numpy(float)
        theta = df_sub.explode("f_theta").f_theta.to_numpy(float)

        h, x = np.histogram(phi, hist_bin(4*18), density=True)
        x = x[:-1] + (x[1] - x[0]) / 2
        x = np.append(np.concatenate((x, x + np.pi), axis=0), x[0])
        h = np.append(np.concatenate((h, h), axis=0), h[0])
        h = h / np.max(h)
        h = h[(x > np.pi * 3 / 2) | (x < np.pi / 2)]
        x = x[(x > np.pi * 3 / 2) | (x < np.pi / 2)]
        axs[i, j].plot(x, h)

        h, x = np.histogram(theta, hist_bin(4*18), density=True)
        x = np.pi - (np.pi / 2 - x[:-1] +
                     (x[1] - x[0]) / 2)  # np/2 for incl, - for plot
        h = h / np.max(h)
        h = h[x > np.pi / 2]
        x = x[x > np.pi / 2]
        axs[i, j].plot(x, h)
        
        phi = df_sub.explode("epa_dir").epa_dir.to_numpy(float)
        
        h, x = np.histogram(phi, hist_bin(2*18), density=True)
        x = x[:-1] + (x[1] - x[0]) / 2
        x = np.append(np.concatenate((x, x + np.pi), axis=0), x[0])
        h = np.append(np.concatenate((h, h), axis=0), h[0])
        h = h / np.max(h)
        h = h[(x > np.pi * 3 / 2) | (x < np.pi / 2)]
        x = x[(x > np.pi * 3 / 2) | (x < np.pi / 2)]
        axs[i, j].plot(x, h, '--')

#         h, x = np.histogram(theta, hist_bin(18), density=True)
#         x = np.pi - (np.pi / 2 - x[:-1] +
#                      (x[1] - x[0]) / 2)  # np/2 for incl, - for plot
#         h = h / np.max(h)
#         h = h[x > np.pi / 2]
#         x = x[x > np.pi / 2]
#         axs[i, j].plot(x, h)